In [1]:
import json
import numpy as np
import pandas as pd
import re
import unicodedata
import string

In [2]:
file = json.loads(open("dev-v2.0.json").read())

In [5]:
pd.json_normalize(file,['data','paragraphs'])

,qas,context
0,[{'question': 'In what country is Normandy loc...,The Normans (Norman: Nourmands; French: Norman...
1,[{'question': 'Who was the duke in the battle ...,"The Norman dynasty had a major political, cult..."
2,[{'question': 'What is the original meaning of...,"The English name ""Normans"" comes from the Fren..."
3,[{'question': 'When was the Duchy of Normandy ...,"In the course of the 10th century, the initial..."
4,[{'question': 'Who upon arriving gave the orig...,"Before Rollo's arrival, its populations did no..."
...,...,...
1199,[{'question': 'Where does centripetal force go...,"where is the mass of the object, is the velo..."
1200,[{'question': 'What is the only form potential...,A conservative force that acts on a closed sys...
1201,[{'question': 'What is sometimes impossible to...,"For certain physical scenarios, it is impossib..."
1202,[{'question': 'In what treatment are nonconser...,The connection between macroscopic nonconserva...


In [6]:
pd.json_normalize(file,['data','paragraphs']).iloc[0]['qas']

[{'question': 'In what country is Normandy located?',
  'id': '56ddde6b9a695914005b9628',
  'answers': [{'text': 'France', 'answer_start': 159},
   {'text': 'France', 'answer_start': 159},
   {'text': 'France', 'answer_start': 159},
   {'text': 'France', 'answer_start': 159}],
  'is_impossible': False},
 {'question': 'When were the Normans in Normandy?',
  'id': '56ddde6b9a695914005b9629',
  'answers': [{'text': '10th and 11th centuries', 'answer_start': 94},
   {'text': 'in the 10th and 11th centuries', 'answer_start': 87},
   {'text': '10th and 11th centuries', 'answer_start': 94},
   {'text': '10th and 11th centuries', 'answer_start': 94}],
  'is_impossible': False},
 {'question': 'From which countries did the Norse originate?',
  'id': '56ddde6b9a695914005b962a',
  'answers': [{'text': 'Denmark, Iceland and Norway', 'answer_start': 256},
   {'text': 'Denmark, Iceland and Norway', 'answer_start': 256},
   {'text': 'Denmark, Iceland and Norway', 'answer_start': 256},
   {'text': 'Den

In [7]:
def read_squad_json(file):
    # read json file
    file = json.loads(open(file).read())
    #get context column and questions column
    context_questions = pd.json_normalize(file,['data','paragraphs'])
    #repeat context column for each question
    idx = np.repeat(context_questions['context'].values, context_questions.qas.str.len())
    #separate question and answer columns
    dataframe = pd.json_normalize(file,record_path=['data','paragraphs','qas'])
    #add context to each question
    dataframe['context'] = idx
    return dataframe

In [94]:
data = read_squad_json("dev-v2.0.json")
data

,question,id,answers,is_impossible,plausible_answers,context
0,In what country is Normandy located?,56ddde6b9a695914005b9628,"[{'text': 'France', 'answer_start': 159}, {'te...",False,NaN,The Normans (Norman: Nourmands; French: Norman...
1,When were the Normans in Normandy?,56ddde6b9a695914005b9629,"[{'text': '10th and 11th centuries', 'answer_s...",False,NaN,The Normans (Norman: Nourmands; French: Norman...
2,From which countries did the Norse originate?,56ddde6b9a695914005b962a,"[{'text': 'Denmark, Iceland and Norway', 'answ...",False,NaN,The Normans (Norman: Nourmands; French: Norman...
3,Who was the Norse leader?,56ddde6b9a695914005b962b,"[{'text': 'Rollo', 'answer_start': 308}, {'tex...",False,NaN,The Normans (Norman: Nourmands; French: Norman...
4,What century did the Normans first gain their ...,56ddde6b9a695914005b962c,"[{'text': '10th century', 'answer_start': 671}...",False,NaN,The Normans (Norman: Nourmands; French: Norman...
...,...,...,...,...,...,...
11868,What is the seldom used force unit equal to on...,5737aafd1c456719005744ff,"[{'text': 'sthène', 'answer_start': 665}, {'te...",False,NaN,"The pound-force has a metric counterpart, less..."
11869,What does not have a metric counterpart?,5ad28ad0d7d075001a4299cc,[],True,"[{'text': 'pound-force', 'answer_start': 4}]","The pound-force has a metric counterpart, less..."
11870,What is the force exerted by standard gravity ...,5ad28ad0d7d075001a4299cd,[],True,"[{'text': 'kilogram-force', 'answer_start': 82}]","The pound-force has a metric counterpart, less..."
11871,What force leads to a commonly used unit of mass?,5ad28ad0d7d075001a4299ce,[],True,"[{'text': 'kilogram-force', 'answer_start': 195}]","The pound-force has a metric counterpart, less..."


In [95]:
def get_text(row):
    answer = ''
    plausible_answers =''
    if row['answers']:
        answer = row['answers'][0]['text']
    if row['plausible_answers']:
        plausible_answers= row['plausible_answers'][0]['text']
    return answer +plausible_answers
# def answer_as_text(dataframe):
#     dataframe['plausible_answers'].fillna('',inplace=True)
#     dataframe['answer_text'] = dataframe.apply(get_text,axis=1)
#     # dataframe['answer_text'] = 'sss '+ dataframe['answer_text'] + ' eee'
#     return dataframe

In [100]:
def answers_begin(row):
    if row['answers']:
        return  row['answers'][0]['answer_start']
    if row['plausible_answers']:
        return row['plausible_answers'][0]['answer_start']
def answers(dataframe):
    # dataframe['plausible_answers'].fillna('',inplace=True)
    dataframe['plausible_answers'].fillna('',inplace=True)
    dataframe['answer_text'] = dataframe.apply(get_text,axis=1)
    dataframe['answer_start'] = dataframe.apply(answers_begin,axis=1)
    dataframe['answer_start'].dropna(inplace=True)
    dataframe['answer_start'] = dataframe['answer_start'].round().astype('Int32')
    return dataframe

In [101]:
ss = '''Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion '''
len(ss)

177

In [102]:
dd['context'][0][159:]

'France. They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries.'

In [103]:
dd = answers(data)
dd

,question,id,answers,is_impossible,plausible_answers,context,answer_text,answer_start,answer_end
0,In what country is Normandy located?,56ddde6b9a695914005b9628,"[{'text': 'France', 'answer_start': 159}, {'te...",False,,The Normans (Norman: Nourmands; French: Norman...,France,159,164
1,When were the Normans in Normandy?,56ddde6b9a695914005b9629,"[{'text': '10th and 11th centuries', 'answer_s...",False,,The Normans (Norman: Nourmands; French: Norman...,10th and 11th centuries,94,116
2,From which countries did the Norse originate?,56ddde6b9a695914005b962a,"[{'text': 'Denmark, Iceland and Norway', 'answ...",False,,The Normans (Norman: Nourmands; French: Norman...,"Denmark, Iceland and Norway",256,282
3,Who was the Norse leader?,56ddde6b9a695914005b962b,"[{'text': 'Rollo', 'answer_start': 308}, {'tex...",False,,The Normans (Norman: Nourmands; French: Norman...,Rollo,308,312
4,What century did the Normans first gain their ...,56ddde6b9a695914005b962c,"[{'text': '10th century', 'answer_start': 671}...",False,,The Normans (Norman: Nourmands; French: Norman...,10th century,671,682
...,...,...,...,...,...,...,...,...,...
11868,What is the seldom used force unit equal to on...,5737aafd1c456719005744ff,"[{'text': 'sthène', 'answer_start': 665}, {'te...",False,,"The pound-force has a metric counterpart, less...",sthène,665,670
11869,What does not have a metric counterpart?,5ad28ad0d7d075001a4299cc,[],True,"[{'text': 'pound-force', 'answer_start': 4}]","The pound-force has a metric counterpart, less...",pound-force,4,14
11870,What is the force exerted by standard gravity ...,5ad28ad0d7d075001a4299cd,[],True,"[{'text': 'kilogram-force', 'answer_start': 82}]","The pound-force has a metric counterpart, less...",kilogram-force,82,95
11871,What force leads to a commonly used unit of mass?,5ad28ad0d7d075001a4299ce,[],True,"[{'text': 'kilogram-force', 'answer_start': 195}]","The pound-force has a metric counterpart, less...",kilogram-force,195,208


0        159
1         94
2        256
3        308
4        671
        ... 
11868    665
11869      4
11870     82
11871    195
11872    383
Name: allanswers, Length: 11873, dtype: Int32

In [39]:
data = answer_as_text(data)[['question','context','answer_text']]

In [40]:
data

,question,context,answer_text
0,In what country is Normandy located?,The Normans (Norman: Nourmands; French: Norman...,France
1,When were the Normans in Normandy?,The Normans (Norman: Nourmands; French: Norman...,10th and 11th centuries
2,From which countries did the Norse originate?,The Normans (Norman: Nourmands; French: Norman...,"Denmark, Iceland and Norway"
3,Who was the Norse leader?,The Normans (Norman: Nourmands; French: Norman...,Rollo
4,What century did the Normans first gain their ...,The Normans (Norman: Nourmands; French: Norman...,10th century
...,...,...,...
11868,What is the seldom used force unit equal to on...,"The pound-force has a metric counterpart, less...",sthène
11869,What does not have a metric counterpart?,"The pound-force has a metric counterpart, less...",pound-force
11870,What is the force exerted by standard gravity ...,"The pound-force has a metric counterpart, less...",kilogram-force
11871,What force leads to a commonly used unit of mass?,"The pound-force has a metric counterpart, less...",kilogram-force


In [10]:
print('avg length of question is :',data['question'].str.split(' ').map(len).mean())
data.iloc[data['question'].str.split(' ').map(len).argmax()]['question']

avg length of question is : 10.090794239029732


"In which case did the Court of Justice state that refusal to admit a lawyer to the Belgian bar because he did not have Belgian heritage wasn't able to be justified?"

In [11]:
print('avg length of context is :',data['context'].str.split(' ').map(len).mean())
data.iloc[data['context'].str.split(' ').map(len).argmax()]['context']

avg length of context is : 127.87256801145456


'The "freedom to provide services" under TFEU article 56 applies to people who give services "for remuneration", especially commercial or professional activity. For example, in Van Binsbergen v Bestuur van de Bedrijfvereniging voor de Metaalnijverheid a Dutch lawyer moved to Belgium while advising a client in a social security case, and was told he could not continue because Dutch law said only people established in the Netherlands could give legal advice. The Court of Justice held that the freedom to provide services applied, it was directly effective, and the rule was probably unjustified: having an address in the member state would be enough to pursue the legitimate aim of good administration of justice. The Court of Justice has held that secondary education falls outside the scope of article 56, because usually the state funds it, though higher education does not. Health care generally counts as a service. In Geraets-Smits v Stichting Ziekenfonds Mrs Geraets-Smits claimed she shoul

In [12]:
print('avg length of answers is :',data['answer_text'].str.split(' ').map(len).mean(), ' word ')
data.iloc[data['answer_text'].str.split(' ').map(len).argmax()]['answer_text']

avg length of answers is : 5.337909542659816  word 


'sss a mouth that can usually be closed by muscles; a pharynx ("throat"); a wider area in the center that acts as a stomach; and a system of internal canals. eee'

# clean text

In [13]:

def remove_punc(text):
    return text.translate(str.maketrans('','',string.punctuation))#'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'
def lower_case(text):
    return text.lower()
def remove_non_english(text):#remove non ascci word
    return unicodedata.normalize('NFKD',text).encode('ascii','ignore').decode('utf-8','ignore')
def remove_links(text):
    return re.sub(r'https?://\S+','',text)
def remove_whitespaces(text):
    return text.strip()

def clean_text(text):
    return remove_punc(lower_case(remove_non_english((remove_links(text)))))

In [14]:
data['question'] = data['question'].apply(clean_text)
data['context'] = data['context'].apply(clean_text)
data['answer_text'] = data['answer_text'].apply(clean_text)

C:\Users\ahmed\AppData\Local\Temp\ipykernel_28528\1711182541.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['question'] = data['question'].apply(clean_text)
C:\Users\ahmed\AppData\Local\Temp\ipykernel_28528\1711182541.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['context'] = data['context'].apply(clean_text)
C:\Users\ahmed\AppData\Local\Temp\ipykernel_28528\1711182541.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_in

In [15]:
data['q_c']='CLS '+data['question']+' SEP '+data['context']

C:\Users\ahmed\AppData\Local\Temp\ipykernel_28528\1210320798.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['q_c']='CLS '+data['question']+' SEP '+data['context']


In [16]:
print('max length of answers is :',data['q_c'].str.split(' ').map(len).max(), ' word ')
data.iloc[data['q_c'].str.split(' ').map(len).argmax()]['q_c']

avg length of answers is : 653  word 


'CLS why was the dutch lawyer who moved to belgium while advising a client in a social society case told he couldnt continue SEP the freedom to provide services under tfeu article 56 applies to people who give services for remuneration especially commercial or professional activity for example in van binsbergen v bestuur van de bedrijfvereniging voor de metaalnijverheid a dutch lawyer moved to belgium while advising a client in a social security case and was told he could not continue because dutch law said only people established in the netherlands could give legal advice the court of justice held that the freedom to provide services applied it was directly effective and the rule was probably unjustified having an address in the member state would be enough to pursue the legitimate aim of good administration of justice the court of justice has held that secondary education falls outside the scope of article 56 because usually the state funds it though higher education does not health 

In [17]:
pd.to_pickle(data,'data.pkl')

In [18]:
from tensorflow.keras.layers import TextVectorization
import tensorflow as tf

In [19]:
#number of words in vocabulary
vocab_size = 40000
# question_vectorizer = TextVectorization(max_tokens=vocab_size,output_sequence_length=15,output_mode='int')
# context_vectorizer = TextVectorization(max_tokens=vocab_size,output_sequence_length=150,output_mode='int')
question_context_vectorizer = TextVectorization(output_sequence_length=200,output_mode='int',standardize=None)# truncate post
answer_vectorizer = TextVectorization(output_sequence_length=5,output_mode='int',standardize=None)

In [20]:
all_words = data['q_c']+data['answer_text']

In [21]:
# question_vectorizer.adapt(all_words)
# context_vectorizer.adapt(all_words)
question_context_vectorizer.adapt(all_words)
answer_vectorizer.adapt(all_words)


In [22]:
#number of words in vocabulary
len(answer_vectorizer.get_vocabulary())

20293

In [23]:
# vectroized_question = question_vectorizer(data['question'])
# vectroized_context = context_vectorizer(data['context'])
vectroized_question_context = question_context_vectorizer(data['q_c']) 
vectroized_answer = answer_vectorizer(data['answer_text'])

In [24]:
print(vectroized_answer.shape)
vectroized_answer[20]

(11873, 5)


<tf.Tensor: shape=(5,), dtype=int64, numpy=array([3068, 6078,  108,   14,    0], dtype=int64)>

In [25]:
dataset = tf.data.Dataset.from_tensor_slices((vectroized_question_context,vectroized_answer))
dataset = dataset.cache()
dataset = dataset.shuffle(buffer_size=len(data))
dataset = dataset.batch(batch_size=32)
dataset = dataset.prefetch(8) #helps prevent bottlenecks

In [26]:
vectroized_question_context.shape

TensorShape([11873, 200])

In [27]:
batch_X ,batch_y = dataset.as_numpy_iterator().next()

In [28]:
batch_X.shape

(32, 200)

In [29]:
train = dataset.take(int(len(data)*0.8))
val = dataset.skip(int(len(data)*0.8)).take(int(len(data)*0.2))

In [30]:
train

<TakeDataset element_spec=(TensorSpec(shape=(None, 200), dtype=tf.int64, name=None), TensorSpec(shape=(None, 5), dtype=tf.int64, name=None))>

In [31]:
train_gen = train.as_numpy_iterator()
train_gen.next()

(array([[   16,    46,    13, ...,     0,     0,     0],
        [   16,    68,    13, ...,     0,     0,     0],
        [   16,    21,    58, ...,     0,     0,     0],
        ...,
        [   16,    85,    41, ...,   999, 15830,   168],
        [   16,    21,  1666, ...,     0,     0,     0],
        [   16,    75, 18344, ...,     0,     0,     0]], dtype=int64),
 array([[ 3068,  1947, 11038,    14,     0],
        [ 3068,     2, 20206,    14,     0],
        [ 3068,     2,   481,     3,  1120],
        [ 3068,  1312,     2,  1614,   351],
        [ 3068,  5932,    14,     0,     0],
        [ 3068,    20,     2,   369, 11450],
        [ 3068,  6974,   389,    14,     0],
        [ 3068,  5687,   965,    14,     0],
        [ 3068,  7756,  1669,    14,     0],
        [ 3068,   312,  2126,    14,     0],
        [ 3068,   913,   656,    14,     0],
        [ 3068,  3047,  1946,  3962,  2436],
        [ 3068,  6005,    32,  1617,    14],
        [ 3068,  1736,  2122,    14,     0],
